## Evaluate ESG scores for bonds portfolio

In [8]:
# load imports
import pandas as pd
import json, math
import plotly.graph_objects as go

In [9]:
# execute the helper functions defined for accessing RDP REST API calls 
%run RDPDefines.ipynb

### What is Symbology Mapping

Symbology Endpoint: [api.refinitiv.com/discovery/symbology/v1/lookup]()

parameters [route = FindESGStatementParent]()

<br/>
Examples:
<br/>
<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 44654483026        |         |
| Issuer     | SAUDI ELECTRICITY GLOBAL SUKUK COMPANY 3        |         |
| 1st Parent | 4295887339 (Saudi Electricity Company)        |    ✔     |
| Result     | 4295887339        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192814833479        |         |
| Issuer     | GOLDMAN SACHS FINANCE CORP INTERNATIONAL LTD        |         |
| 1st Parent | GS Global Markets Inc        |         |
| 2nd Parent | 4295911963 (Goldman Sachs Group Inc)        |    ✔   |
| Result     | 4295911963       |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192846098875        |         |
| Issuer     | MORGAN STANLEY BANK NA        |         |
| 1st Parent | MORGAN STANLEY DOMESTIC HOLDINGS INC        |         |
| 2nd Parent | Morgan Stanley Capital Management LLC        |         |
| 3rd Parent | 4295904557 (Morgan Stanley)        |    ✔   |
| Result     | 4295904557        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 46641173275        |         |
| Issuer     | PROPERTY AND BUILDING CORP LTD        |         |
| 1st Parent | DISCOUNT INVESTMENT CORP LTD        |         |
| 2nd Parent | DOLPHIN NETHERLANDS BV        |         |
| 3rd Parent | TYRUS SA        |         |
| 4th Parent | 5000620306 (RSA INVERSIONES Y REPRESENTACIONES SA)        |    ✔    |
| Result     | 5000620306        |         |


### Download the bulk JSON files

In [10]:
# define the download function
def downloadJSONBulkFile(bucketName, fileAttributes, fileNameKeywords):
    # get a list of all the buckets
    hResp = getRequest('/file-store/v1/file-sets?bucket=' + bucketName + '&pageSize=100&attributes=' + fileAttributes)
    print(hResp)
    # loop through all the buckets
    for bucket in hResp['value']:
        bName = bucket['name']
        # does bucket contains all the matching keywords
        if all([x in bName for x in fileNameKeywords]):
            fileName = bucket['files'][0]
            print('Found bucket: ', bName, ', FileName: ', fileName)
            # stop any more searching
            break
    
    if not fileName:
        raise Exception('No matching bulk file found in bucket:'.format(bucketName))

    # download and uncompress the file object
    fileStr = downloadUncompressFile('/file-store/v1/files/' + fileName + '/stream')
    print('File downloaded and uncompressed, size: ', len(fileStr))
    return fileStr


#### Download and save the Bond ISIN - ESG Parent mapping

In [4]:
# download the Bond-ESGParent symbology database
jsonlFile = downloadJSONBulkFile('bulk-symbology', 'ContentType:Symbology BondISINSusFinMapping', ['Bond', 'ISIN', 'Json', 'Init'])
# parse out the entries in the bulk file
mapping = []
for l in jsonlFile.splitlines():
    jObj = json.loads(l)
    if len(jObj['Identifiers']) > 0 and jObj['EsgCoverage']['EsgStatementParentOrganization']:
        coName = jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'][0]['NormalizedName'] if jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'] else ''
        mapping.append((jObj['Identifiers'][0]['IdentifierValue'], jObj['EsgCoverage']['EsgStatementParentOrganization']['ObjectId'], coName))
    
print('Loaded {} Bonds ISIN to ESG Parent PermID mappings'.format(len(mapping)))

Getting access token...
...token received
{'value': [{'id': '4018-d7c1-b8574d07-bd57-ae61e105c79f', 'name': 'Bulk-Global-BondISINSusFinMapping-v1-Csv-Init-2023-05-07T16:01:06.616Z', 'bucketName': 'bulk-Symbology', 'packageId': '4edd-8742-9ee36c59-af01-9d2166299ba6', 'attributes': [{'name': 'ContentType', 'value': 'Symbology BondISINSusFinMapping'}], 'files': ['4122-50bf-eadac63b-a83e-42af614ea901', '4331-df2d-f9f8e2d1-80ef-c817c86fea49', '4384-0ffd-a981ba8f-bf8d-44d37482573f', '4d2e-14d0-c00da7f9-8952-54b4c83b10ad', '4e2a-9d4b-1ac03f10-bc81-5488cf3a7f16'], 'numFiles': 5, 'contentFrom': '1970-01-01T00:00:00Z', 'contentTo': '2023-05-07T15:55:00Z', 'availableFrom': '2023-05-07T17:03:42Z', 'availableTo': '2023-08-07T17:03:42Z', 'status': 'READY', 'created': '2023-05-07T17:03:42Z', 'modified': '2023-05-07T17:04:09Z'}, {'id': '40ee-6a26-720c2490-8d1b-9382fa738ffa', 'name': 'Bulk-Global-BondISINSusFinMapping-v1-Csv-Init-2023-04-23T16:01:49.957Z', 'bucketName': 'bulk-Symbology', 'packageId': '

In [5]:
# load the dataset into a pandas dataframe
df1 = pd.DataFrame(mapping, columns=['Bond', 'ESGParent', 'ParentName'])
# save the database
df1.to_pickle('Bond_Parent_mapping.pkl')

#### Download and save the ESG Scores dataset

In [21]:
# download the ESG Scores database
jsonlFile = downloadJSONBulkFile('bulk-ESG', 'ContentType:ESG Scores', ['Scores-Full', 'Init', 'Jsonl'])

{'value': [{'id': '4009-4877-b3e35682-8ede-0ee6b0555b85', 'name': 'Bulk-ESG-Global-Scores-Wealth-Standard-v1-Jsonl-Delta-2023-05-21T17:21:12.117Z', 'bucketName': 'bulk-ESG', 'packageId': '4d07-921a-7337a7cc-a291-e5b049ddf4dc', 'attributes': [{'name': 'ContentType', 'value': 'ESG Scores'}], 'files': ['4af3-ea1a-124866b4-b058-8423fae37128'], 'numFiles': 1, 'contentFrom': '2023-05-14T17:15:00Z', 'contentTo': '2023-05-21T17:15:00Z', 'availableFrom': '2023-05-21T17:23:42Z', 'availableTo': '2023-06-04T17:23:41Z', 'status': 'READY', 'created': '2023-05-21T17:23:42Z', 'modified': '2023-05-21T17:24:06Z'}, {'id': '4183-acd4-05797f0e-8645-274e69d0b094', 'name': 'Bulk-ESG-Global-Scores-Current-v1-Jsonl-Init-2023-05-21T17:10:52.817Z', 'bucketName': 'bulk-ESG', 'packageId': '4ad7-60f2-9bd6fd87-b3b2-c68a28a2daca', 'attributes': [{'name': 'ContentType', 'value': 'ESG Scores'}], 'files': ['4297-7712-3037e9e4-a9e5-4bc520f67f55'], 'numFiles': 1, 'contentFrom': '1970-01-01T00:00:00Z', 'contentTo': '2023-0

In [22]:
scores = []
for l in jsonlFile.splitlines():
    j = json.loads(l)
    e = j['ESGScores']
    scores.append((j['StatementDetails']['OrganizationId'],
        j['StatementDetails']['FinancialPeriodFiscalYear'],
        e['ESGCombinedScore']['Value'], 
        e['ESGScore']['Value'],
        e['EnvironmentPillarScore']['Value'],
        e['ESGResourceUseScore']['Value'],
        e['ESGEmissionsScore']['Value'],
        e['ESGInnovationScore']['Value'],
        e['SocialPillarScore']['Value'],
        e['ESGWorkforceScore']['Value'],
        e['ESGHumanRightsScore']['Value'],
        e['ESGCommunityScore']['Value'],
        e['ESGProductResponsibilityScore']['Value'],
        e['GovernancePillarScore']['Value'],
        e['ESGManagementScore']['Value'],
        e['ESGShareholdersScore']['Value'],
        e['ESGCsrStrategyScore']['Value'],
        e['ESGCControversiesScore']['Value']))


print('Loaded {} scores'.format(len(scores))) 


Loaded 110908 scores


In [23]:
# load the dataset into a pandas dataframe
df2 = pd.DataFrame(scores, columns=['OrganizationId', 'FiscalYear', 'ESGCombinedScore', 'ESGScore', 'EnvironmentPillarScore', 'ESGResourceUseScore', 'ESGEmissionsScore', 'ESGInnovationScore', 'SocialPillarScore', 'ESGWorkforceScore', 'ESGHumanRightsScore', 'ESGCommunityScore', 'ESGProductResponsibilityScore', 'GovernancePillarScore', 'ESGManagementScore', 'ESGShareholdersScore', 'ESGCsrStrategyScore', 'ESGCControversiesScore'])
# change the Fiscal Year data type to a number
df2['FiscalYear'] = df2['FiscalYear'].astype(int)
# keep the latest ESG scores only
df2 = df2.loc[df2.groupby(['OrganizationId'])['FiscalYear'].idxmax()].reset_index(drop=True)
# save the database
df2.to_pickle('ESGScores.pkl')

### Load the pre-downloaded database for Symbology mapping and ESG

In [24]:
bMapping = pd.read_pickle('Bond_Parent_mapping.pkl')
bMapping.head()

,Bond,ESGParent,ParentName
0,KR6HN0001YR8,4295882718,Hana Financial Group Inc
1,KR6703304A47,4295882718,Hana Financial Group Inc
2,US78016FGF53,8589934213,Royal Bank of Canada
3,XS2291434251,4297375292,Marex Financial
4,XS2531033020,4295888106,DBS Group Holdings Ltd


In [25]:
scores = pd.read_pickle('ESGScores.pkl').astype({'ESGCombinedScore': float, 'ESGScore': float, 'EnvironmentPillarScore': float, 'ESGResourceUseScore': float, 'ESGEmissionsScore': float, 'ESGInnovationScore': float, 'SocialPillarScore': float, 'ESGWorkforceScore': float, 'ESGHumanRightsScore': float, 'ESGCommunityScore': float, 'ESGProductResponsibilityScore': float, 'GovernancePillarScore': float, 'ESGManagementScore': float, 'ESGShareholdersScore': float, 'ESGCsrStrategyScore': float, 'ESGCControversiesScore': float})
scores.head()

,OrganizationId,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,ESGInnovationScore,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,4295533401,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4295613014,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4295641240,2022,0.643688,0.643688,0.438281,0.743781,0.690594,0.0,0.835678,0.608597,0.831551,0.970588,0.969340,0.573393,0.457340,0.970263,0.558349,1.0
3,4295856018,2021,0.163130,0.163130,0.086378,0.000000,0.188462,0.0,0.201966,0.221875,0.107477,0.581250,0.000000,0.162608,0.153226,0.250000,0.078431,1.0
4,4295856019,2022,0.134542,0.134542,0.000000,0.000000,0.000000,0.0,0.126409,0.007862,0.000000,0.009434,0.818339,0.346373,0.258333,0.825000,0.068627,1.0


### Get the Bond portfolio holdings

In [26]:
# what is the Lipper ID of the bonds portfolio
portfolioID = 60000170

In [27]:
# get the constituents bonds in this portfolio
hResp = getRequest('/data/funds/v1/assets/' + str(portfolioID), {'properties': 'holdings'})
print(hResp)

{'assets': [{'id': '60000170', 'holdings': [{'date': '2023-04-30', 'securitiesHeldCount': 1167, 'constituents': [{'name': 'UNITED STATES OF AMERICA (GOVERNMENT) 6.25% 15-MAY-2030', 'country': 'UNITED STATES', 'weight': 4.5449, 'weightPrevious': 4.6235, 'weightChange': -0.0786, 'type': {'id': '13737', 'code': 'SOVEREIGN BOND', 'name': 'Sovereign Bond'}, 'sharesHeld': 747077200.0, 'sharesPrevious': 747077200.0, 'sharesChange': 0.0, 'crossReferenceCodes': [{'code': 'RIC', 'type': {'id': '26', 'code': 'RIC', 'name': 'RIC'}, 'values': [{'value': '912810FM5='}]}, {'code': 'ISIN', 'type': {'id': '1424', 'code': 'ISIN', 'name': 'ISIN Code'}, 'values': [{'value': 'US912810FM54'}]}, {'code': 'CUSIP', 'type': {'id': '4862', 'code': 'CUSIP', 'name': 'CUSIP'}, 'values': [{'value': '912810FM5'}]}, {'code': 'SEDOL', 'type': {'id': '1420', 'code': 'SEDOL', 'name': 'Sedol Code'}, 'values': [{'value': 'B7T79C5'}]}], 'rank': 1, 'marketValue': 872446092.6, 'marketValueCurrency': 'USD', 'maturityDate': '20

In [28]:
allHoldings = []
# extract the ISIN, and weights of the bond holdings
for a in hResp['assets'][0]['holdings'][0]['constituents']:
    if 'crossReferenceCodes' in a:
        for code in a['crossReferenceCodes']:
            if code['code'] == 'ISIN':
                allHoldings.append((code['values'][0]['value'], a['weight']))

display(allHoldings[:10])
print('This fund contains {} bonds'.format(len(allHoldings)))

[('US912810FM54', 4.5449),
 ('US912810FJ26', 3.1941),
 ('US91282CGP05', 2.5222),
 ('US91282CGH88', 2.517),
 ('US31359MGK36', 2.2238),
 ('US912810FB99', 1.9792),
 ('US91282CFV81', 1.6848),
 ('LU1900232734', 1.4624),
 ('US912810ET17', 1.1968),
 ('US21H0306587', 1.0709)]

This fund contains 1156 bonds


### Match the ESG-Parent company of these bonds

In [29]:
# create a master dataframe for all processing
mdf = pd.DataFrame(allHoldings, columns =['Bond', 'Weight'])
# merge the ESG parent company info into this dataframe
mdf = mdf.merge(bMapping, how='left', left_on='Bond', right_on='Bond')
display(mdf)

,Bond,Weight,ESGParent,ParentName
0,US912810FM54,4.5449,NaN,NaN
1,US912810FJ26,3.1941,NaN,NaN
2,US91282CGP05,2.5222,NaN,NaN
3,US91282CGH88,2.5170,NaN,NaN
4,US31359MGK36,2.2238,4295903973,Federal National Mortgage Association
...,...,...,...,...
1151,US12594CBG42,0.0001,NaN,NaN
1152,XS2406881669,0.0000,4295865535,Yango Group Co Ltd
1153,USL9116PAG83,0.0000,NaN,NaN
1154,US92824BAA44,0.0000,NaN,NaN


In [30]:
total = len(allHoldings)
covered = len(mdf['ESGParent'].dropna())
coverage = (covered / total) * 100
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = coverage,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': 'Coverage % ({} out of {} have ESG data)'.format(covered, total) },
    gauge = {'axis': {'range': [None, 100]}}))

fig.show()

### Calculate and display the consolidated ESG Score for the whole portfolio

In [31]:
# formulate everything onto a dataframe and display
combined = mdf.merge(scores, how='left', left_on='ESGParent', right_on='OrganizationId')
combined.drop('OrganizationId', axis=1, inplace=True)
display(combined)

,Bond,Weight,ESGParent,ParentName,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,...,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,US912810FM54,4.5449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US912810FJ26,3.1941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US91282CGP05,2.5222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US91282CGH88,2.5170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US31359MGK36,2.2238,4295903973,Federal National Mortgage Association,2022.0,0.540331,0.540331,0.434402,0.271028,0.091904,...,0.499387,0.568781,0.000000,0.600906,0.754333,0.639113,0.850063,0.314053,0.071956,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,US12594CBG42,0.0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1152,XS2406881669,0.0000,4295865535,Yango Group Co Ltd,2021.0,0.458041,0.458041,0.384204,0.478149,0.111979,...,0.285378,0.262766,0.195082,0.158511,0.777641,0.719345,0.731657,0.918796,0.358605,1.0
1153,USL9116PAG83,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,US92824BAA44,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Rebase, calculate the combined ESG scores of these holdings
weightedSeries = []
for idx, a in combined['ESGCombinedScore'].items():
    if math.isnan(a):
        weightedSeries.append(0)
    else:
        weightedSeries.append(combined['Weight'][idx])

weightTotal = sum(weightedSeries)
rebasedWeight = combined['Weight']/weightTotal

In [33]:
# calculate the weighted total for the holdings
total = []
for col in combined:
    if col == 'Bond':
        total.append('WEIGHTED AVERAGE')
    elif col == 'Weight':
        total.append(1.0)
    elif col == 'FiscalYear':
        total.append('')
    elif combined[col].dtype == 'float64':
        total.append((combined[col] * rebasedWeight).sum())
    else:
        total.append('')

In [34]:
# insert the final result into the portfolio
combined.loc[-1] = total
combined.index = combined.index + 1
combined = combined.sort_index()

In [35]:
# display the final dataframe
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option("display.precision", 2)
display(combined.fillna(''))

,Bond,Weight,ESGParent,ParentName,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,ESGInnovationScore,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,WEIGHTED AVERAGE,1.0000,,,,0.541238,0.63078,0.572596,0.608827,0.606727,0.424547,0.64045,0.691415,0.52506,0.734269,0.571985,0.654299,0.688324,0.581843,0.59286,0.705028
1,US912810FM54,4.5449,,,,,,,,,,,,,,,,,,,
2,US912810FJ26,3.1941,,,,,,,,,,,,,,,,,,,
3,US91282CGP05,2.5222,,,,,,,,,,,,,,,,,,,
4,US91282CGH88,2.5170,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,US12594CBG42,0.0001,,,,,,,,,,,,,,,,,,,
1153,XS2406881669,0.0000,4295865535,Yango Group Co Ltd,2021.0,0.458041,0.458041,0.384204,0.478149,0.111979,0.66436,0.285378,0.262766,0.195082,0.158511,0.777641,0.719345,0.731657,0.918796,0.358605,1.0
1154,USL9116PAG83,0.0000,,,,,,,,,,,,,,,,,,,
1155,US92824BAA44,0.0000,,,,,,,,,,,,,,,,,,,


In [ ]:
combined.to_clipboard()